<a href="https://colab.research.google.com/github/karchx/spark/blob/main/spark-t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset create CSV with pyspark

In [ ]:
!pip install pyspark pandas
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import requests
import json

data = requests.get("https://www.defense.gov/data.json")
data = data.json()
with open("/tmp/ds_w.json", "w") as f:
  f.write(json.dumps(data["dataset"]))

In [ ]:
# !cat /tmp/ds.json
!ls -l --block-size=M /tmp/ds_w.json

-rw-r--r-- 1 root root 1M Mar 22 01:54 /tmp/ds_w.json


In [ ]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession

#conf = SparkConf().setMaster("local").setAppName("create_csv")

# Inicializo el Spark Context
spark = SparkSession.builder.master("local").appName("create_csv").getOrCreate()

#spark = SparkSession.builder.master("local").appName("create_csv").getOrCreate()
#spark = SQLContext(sc)

In [ ]:
import pyspark.sql.functions as F

df = spark.createDataFrame(data["dataset"])

df2 = df.select("*", F.explode(df.publisher))

pv = df2.select("keyword", "key", "value")\
  .groupBy(["keyword", "key", "value"])\
  .pivot("key").agg(F.first("value"))\

keywords = pv.drop("@type", "key", "value")
keywords.select('name').distinct().show(truncate=False)

+---------------------------------------+
|name                                   |
+---------------------------------------+
|US Army Corps of Engineers             |
|Defense Commissary Agency              |
|Department of Defense Inspector General|
|Office of the Secretary of Defense     |
|Defense Logistics Agency               |
|Federal Voting Assistance Program      |
|Department of Defense                  |
|Department of the Air Force            |
|NULL                                   |
+---------------------------------------+



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql.functions import size

df = spark.read.csv("/content/drive/My Drive/providers.csv", header=True)
df = df.alias("provider")
keywords = keywords.alias("keyword")
join = df.join(keywords, [F.col("keyword.name") == F.col("provider.name")], how="right")
#df_clean = join.dropna(how="any")
#df_clean.show()
join.drop("all").show()
#df.select("publisher.name", "keyword").show()

+--------------------+----+--------------------+--------------------+
|                name|  id|             keyword|                name|
+--------------------+----+--------------------+--------------------+
|                NULL|NULL|[Active, Air Forc...|                NULL|
|Department of Def...|   3|[Active, Air Forc...|Department of Def...|
|                NULL|NULL|[Air Force, Alaba...|                NULL|
|Department of Def...|   3|[Air Force, Alaba...|Department of Def...|
|                NULL|NULL|[Air Force, Alask...|                NULL|
|Department of Def...|   3|[Air Force, Alask...|Department of Def...|
|                NULL|NULL|      [Cost Savings]|                NULL|
|Department of Def...|   3|      [Cost Savings]|Department of Def...|
|                NULL|NULL|        [DLA Energy]|                NULL|
|Defense Logistics...|   2|        [DLA Energy]|Defense Logistics...|
|                NULL|NULL|[DLA Strategic Plan]|                NULL|
|Defense Logistics..

Goal

Create a silver layer of data that performs some transformations on the bronze data.

This data should provide two new tables:

providers
the unique name of each provider
keywords
how often each keyword occurred for each provider
Requirements
The following data should exist in the silver layer
providers
id: integer
name: string

keywords
id: integer
provider_id: string
keyword: string
count: integer

Any assumptions about the data, process, etc. are captured in the comments on this story
Any additional testing or capabilities are captured in the comments on this story
Blockers
This story requires that bronze data already exists